In [5]:
import torch
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
k = 5
n = 10
m = 20

In [ ]:
def prediction(x: torch.Tensor, phi: torch.Tensor, p: torch.Tensor):
    """ retun an y-vector prediction to given x, phi and p.

    Given x is (m, n), phi is (n, k) and p is (k, 1), we perform
    NB prediction on the matrices to get an y-vector prediction.
    For k classes, each element in y_i in y is in [0,k)

    
    """
    logPhi = phi.log()      # n, k
    logP = p.log().T        # 1, k

    scores = x @ logPhi + logP
    return scores.argmax(dim=1)

def generateData(k: int, n: int, m: int):
    # generate real P (probability of y matrix)
    P = torch.randint(low=1, high=6, size=(k,), device=device)
    PSum = P.sum()
    P = P / PSum
    logP = P.log()


    # generate real phi
    phi = torch.randint(low=1, high=5, size=(n, k), device=device)
    phiSum = phi.sum(dim=0, keepdim=True).expand(size=phi.size())
    phi = phi / phiSum
    logPhi = phi.log()


    # generate real phi
    x = torch.randint(low=0, high=2, size=(m, n), device=device)



RuntimeError: expected m1 and m2 to have the same dtype, but got: long int != float

In [ ]:
# generate real P (probability of y matrix)
P = torch.randint(low=1, high=6, size=(k,), device=device)
PSum = P.sum()
P = P / PSum

tensor([0.0667, 0.3333, 0.1333, 0.2667, 0.2000])